In [111]:
import numpy as np
from itertools import product, combinations

#### (Inductive logic programming in the context of prop ositional logic) Le p,q,r be propositional variables (6p)

Since i find it intreating to apply theory to parties i will try to implement the algorithm in python. My argument for that is that if i can implement it then i probably understand it. I will reason for the answers that the algorithm gives.

1. Suppose that all the formulas ¬p ∨ r, ¬q ∨ ¬r and p are true. Determine the
truth values of q and r.

In [71]:
true_statements = [
    lambda p, q, r: not p or r,
    lambda p, q, r: not q or not r,
    lambda p, q, r: p,
]

for p, q, r in product([True, False], repeat=3):
    if all(map(lambda f: f(p, q, r), true_statements)):
        print(f"p={p}, q={q}, r={r}")


p=True, q=False, r=True


* if p is true then r must be true given true statement ¬p ∨ r.
* if r is true then q must be false given true statement ¬q ∨ ¬r.

Answer: q is false and r is true.

2. Construct a CNF φ which avoids N and covers E where:

    N = {(0,1,0),(1,0,0),(1,1,0)} and

    E = {(0,0,0),(0,0,1),(0,1,1),(1,0,1),(1,1,1)}.
    
    Try to simplify φ as much as p ossible by removing unnecessary 1 literals (if such
    exist).

In [177]:
# E = np.array([(1,1,0,1),(1,0,1,0),(0,1,1,0),(0,1,0,1),(1,0,1,1)])
# N = np.array([(0,0,0,0),(1,1,1,1),(1,1,1,0),(0,1,0,0),(1,0,0,1)])
E = np.array([(0,0,0),(0,0,1),(0,1,1),(1,0,1),(1,1,1)])
N = np.array([(0,1,0),(1,0,0),(1,1,0)])
def covers(cause, statements):
    """
    all positive examples satisfy φ
    """
    cause = np.array(cause, dtype=bool)
    covered = np.zeros(len(statements), dtype=bool)
    for i, statement in enumerate(statements):
        res = list(map(lambda c, s: c if s  else not c, cause, statement))
        covered[i] = any(res)
    return all(covered)

def avoids(cause, statements):
    """
    no negative example satisfy φ
    """
    cause = np.array(cause, dtype=bool)
    avoided = np.zeros(len(statements), dtype=bool)
    for i, statement in enumerate(statements):
        res = list(map(lambda c, s: c if s  else not c, cause, statement))
        avoided[i] = all(res)
    return not any(avoided)

def cause_str(cause):
    cause, idxs = cause
    return '(' + ' ∨ '.join(map(lambda i: f'{L[i]}' if cause[i] else f'¬{L[i]}', idxs)) + ')'

L = ['p', 'q', 'r', 's', 't']

def find_H(cause, E, N, maximal=True):
    for cause_size in  sorted(range(1, len(E[0])+1), reverse=maximal):
        for idxs in combinations(range(len(E[0])), cause_size):
            curr_cause = [cause[i] for i in idxs]
            if covers(curr_cause, statements=E[:, idxs]) and avoids(curr_cause, statements=N[:, idxs]):
                return cause, idxs
    return None, None

def algorithm(E, N, maximal=True):
    """
    Algorithm 1; producing a maximally general set or simplified set of clauses H which covers E and avoids N
    (3.3) in Logic_and_probability_7.pdf
    """
    inxs = []
    H = []
    used = set()
    for cause in N:
        cause = [int(not c) for c in cause]
        cause, idxs = find_H(cause, E, N, maximal)
        if cause is not None:
            curr = tuple([cause[i] for i in idxs] + list(idxs))
            if curr in used:
                continue
            used.add(curr)
            H.append(cause)
            inxs.append(idxs)
    return zip(H, inxs)

Fi = algorithm(E, N, maximal=True)
print('Maximally CNF:')
print(' ∧ '.join(map(cause_str, Fi)))

Fi = algorithm(E, N, maximal=False)
print('Simplified CNF:')
print(' ∧ '.join(map(cause_str, Fi)))

Maximally CNF:
(p ∨ ¬q ∨ r) ∧ (¬p ∨ q ∨ r) ∧ (¬p ∨ ¬q ∨ r)
Minimally CNF:
(¬q ∨ r) ∧ (¬p ∨ r)


#### Answer:    
Algorithm 1/2; producing a maximally general set or simplified set of clauses H which covers E and avoids N. (3.3) in Logic_and_probability_7.pdf


1. φ with Maximally CNF: 

    (p ∨ ¬q ∨ r) ∧ (¬p ∨ q ∨ r) ∧ (¬p ∨ ¬q ∨ r)

2. φ with Simplified CNF: 

    (¬q ∨ r) ∧ (¬p ∨ r)


#### In the rest of the assignments L and M refer to the language L and L-structure M that we now define. 

Let L = {P,Q,R,S,T} where the relation symbols P,Q,R,S,T have arities 1,1,2,2,2, respectively.

Let M b e the L-structure with domain M = {1,2,3,...,20} and the following interpretations of the relation symbols:
* PM = {1,2,3,4,5,6,7,8,9,10},
* QM = {1,2,3,4,5,11,12,13,14,15},
* RM = {(i,j) : i ̸= j,1 ≤i ≤5 and 1 ≤j ≤5},
* SM = {(2,5),(3,1),(4,3),(1,2),(5,4),(2,3),(4,2),(1,3),(3,5),(4,5)},
* TM = {(2,5),(3,1),(4,3),(1,2),(5,4),(3,2),(2,1),(1,4),(2,4),(5,2)}.

In [178]:
class Base:
    def __repr__(self) -> str:
        return self.name + '(' + ','.join(map(str, self.args)) + ')'

class P(Base):
    def __init__(self, *args):
        self.name = "P"
        self.args = args
        self.arity = 1
        assert len(args) == self.arity
    
class Q(Base):
    def __init__(self, *args):
        self.name = "Q"
        self.args = args
        self.arity = 1
        assert len(args) == self.arity

class R(Base):
    def __init__(self, *args):
        self.name = "R" 
        self.args = args
        self.arity = 2
        assert len(args) == self.arity

class S(Base):
    def __init__(self, *args):
        self.name = "S"
        self.args = args
        self.arity = 2
        assert len(args) == self.arity

class T(Base):
    def __init__(self, *args):
        self.name = "T"
        self.args = args
        self.arity = 2
        assert len(args) == self.arity
M = list(range(1,21))
PM = list(map(P, [1,2,3,4,5,6,7,8,9,10]))
QM = list(map(Q, [1,2,3,4,5,11,12,13,14,15]))
RM = [R(i,j) for i in range(1,6) for j in range(1,6) if i != j]
SM = list(map(S, [2,3,4,1,5,2,4,1,3,4], [5,1,3,2,4,3,2,3,5,5]))
TM = list(map(T, [2,3,4,1,5,3,2,1,2,5], [5,1,3,2,4,2,1,4,4,2]))
print(PM)
print(QM)
print(RM)
print(SM)
print(TM)


[P(1), P(2), P(3), P(4), P(5), P(6), P(7), P(8), P(9), P(10)]
[Q(1), Q(2), Q(3), Q(4), Q(5), Q(11), Q(12), Q(13), Q(14), Q(15)]
[R(1,2), R(1,3), R(1,4), R(1,5), R(2,1), R(2,3), R(2,4), R(2,5), R(3,1), R(3,2), R(3,4), R(3,5), R(4,1), R(4,2), R(4,3), R(4,5), R(5,1), R(5,2), R(5,3), R(5,4)]
[S(2,5), S(3,1), S(4,3), S(1,2), S(5,4), S(2,3), S(4,2), S(1,3), S(3,5), S(4,5)]
[T(2,5), T(3,1), T(4,3), T(1,2), T(5,4), T(3,2), T(2,1), T(1,4), T(2,4), T(5,2)]


In [186]:
class F(Base):
    arity = 1
    name = "F"
    def __init__(self, *args):
        self.args = args
        assert len(args) == F.arity
class P(Base):
    arity = 2
    name = "P"
    def __init__(self, *args):
        self.args = args
        assert len(args) == P.arity

class D(Base):
    arity = 2
    def __init__(self, *args):
        self.name = "D"
        self.args = args
        assert len(args) == D.arity

class G(Base):
    arity = 2
    name = "G"
    def __init__(self, *args):
        self.args = args
        assert len(args) == G.arity

A = ['a', 'b', 'e', 'g', 'h', 'm', 'n', 't']

relations = {'F': [F('m'), F('h'), F('e'), F('n')], # (i.e. m,n,e, and n are females),
             'P': [P('g', 'm'), P('h', 'm'), P('h', 't'), P('t', 'e'), P('n', 'e'), P('a', 't'), P('e', 'b')], # (i.e. g is a parent of m, h is a parent of m, and so on)
             'D': [D('g', 'm'), D('h', 'm'), D('t', 'e'), D('n', 'e')], # (i.e. g is a daughter of m, h is a daughter of m, and so on)
             'G': [G('h', 'e'), G('n', 'b')], # (i.e. h is a grandparent of e, n is a grandparent of b)
}




In [180]:
def concept_learning(concept, domain, relations, structures, variables=2, consept_size=2):
    for vars in product(domain, repeat=variables):
        statment = concept(*vars[:concept.arity])
        for structure in product(structures, repeat=consept_size):
            for structure_var in combinations(vars, structure.arity):
                if statment(structure_var):
                    print(statment, structure_var)
            

        print(selected)

concept_learning(D, A, relations, [F, P, G])

('a', 'a')
('a', 'b')
('a', 'e')
('a', 'g')
('a', 'h')
('a', 'm')
('a', 'n')
('a', 't')
('b', 'a')
('b', 'b')
('b', 'e')
('b', 'g')
('b', 'h')
('b', 'm')
('b', 'n')
('b', 't')
('e', 'a')
('e', 'b')
('e', 'e')
('e', 'g')
('e', 'h')
('e', 'm')
('e', 'n')
('e', 't')
('g', 'a')
('g', 'b')
('g', 'e')
('g', 'g')
('g', 'h')
('g', 'm')
('g', 'n')
('g', 't')
('h', 'a')
('h', 'b')
('h', 'e')
('h', 'g')
('h', 'h')
('h', 'm')
('h', 'n')
('h', 't')
('m', 'a')
('m', 'b')
('m', 'e')
('m', 'g')
('m', 'h')
('m', 'm')
('m', 'n')
('m', 't')
('n', 'a')
('n', 'b')
('n', 'e')
('n', 'g')
('n', 'h')
('n', 'm')
('n', 'n')
('n', 't')
('t', 'a')
('t', 'b')
('t', 'e')
('t', 'g')
('t', 'h')
('t', 'm')
('t', 'n')
('t', 't')


In [188]:
for selected in product([F, P, G], repeat=3):
    print(selected)
    

(<class '__main__.F'>, <class '__main__.F'>, <class '__main__.F'>)
(<class '__main__.F'>, <class '__main__.F'>, <class '__main__.P'>)
(<class '__main__.F'>, <class '__main__.F'>, <class '__main__.G'>)
(<class '__main__.F'>, <class '__main__.P'>, <class '__main__.F'>)
(<class '__main__.F'>, <class '__main__.P'>, <class '__main__.P'>)
(<class '__main__.F'>, <class '__main__.P'>, <class '__main__.G'>)
(<class '__main__.F'>, <class '__main__.G'>, <class '__main__.F'>)
(<class '__main__.F'>, <class '__main__.G'>, <class '__main__.P'>)
(<class '__main__.F'>, <class '__main__.G'>, <class '__main__.G'>)
(<class '__main__.P'>, <class '__main__.F'>, <class '__main__.F'>)
(<class '__main__.P'>, <class '__main__.F'>, <class '__main__.P'>)
(<class '__main__.P'>, <class '__main__.F'>, <class '__main__.G'>)
(<class '__main__.P'>, <class '__main__.P'>, <class '__main__.F'>)
(<class '__main__.P'>, <class '__main__.P'>, <class '__main__.P'>)
(<class '__main__.P'>, <class '__main__.P'>, <class '__main__.